Примеры кода на Python
Простой пример: Адаптация имени метода
Представим, что у нас есть старая система логирования, которая использует метод log_message, а новая система ожидает метод write_log.

In [1]:
import datetime

# --- Adaptee ---
class OldLogger:
    """Старая система логирования с несовместимым методом."""
    def log_message(self, severity: str, message: str):
        timestamp = datetime.datetime.now().isoformat()
        print(f"[OLD LOG][{severity.upper()}] {timestamp}: {message}")

# --- Target Interface (неявный) ---
# Клиент ожидает объект с методом write_log(level: int, message: str)
# Уровни: 1=INFO, 2=WARNING, 3=ERROR

# --- Adapter ---
class LoggerAdapter:
    """Адаптер, который преобразует вызовы write_log в log_message."""
    def __init__(self, old_logger: OldLogger):
        self._adaptee = old_logger
        self._level_map = {1: "INFO", 2: "WARNING", 3: "ERROR"}

    def write_log(self, level: int, message: str):
        severity = self._level_map.get(level, "UNKNOWN")
        print(f"Adapter: Calling adaptee's log_message for level {level}...")
        self._adaptee.log_message(severity, message)

# --- Client ---
def client_code(logger):
    """Клиентский код, использующий Target интерфейс."""
    print("Client: Writing some logs...")
    logger.write_log(1, "Application started successfully.")
    logger.write_log(2, "Configuration file not found, using defaults.")
    logger.write_log(3, "Critical error encountered!")

# --- Использование ---
old_system_logger = OldLogger()
print("Trying to use OldLogger directly (will fail or not what client expects):")
# client_code(old_system_logger) # Вызовет AttributeError, т.к. нет метода write_log

print("\nUsing OldLogger via Adapter:")
adapter = LoggerAdapter(old_system_logger)
client_code(adapter)

# Вывод:
# Trying to use OldLogger directly (will fail or not what client expects):
#
# Using OldLogger via Adapter:
# Client: Writing some logs...
# Adapter: Calling adaptee's log_message for level 1...
# [OLD LOG][INFO] 2023-10-27T10:30:00.123456: Application started successfully.
# Adapter: Calling adaptee's log_message for level 2...
# [OLD LOG][WARNING] 2023-10-27T10:30:00.123456: Configuration file not found, using defaults.
# Adapter: Calling adaptee's log_message for level 3...
# [OLD LOG][ERROR] 2023-10-27T10:30:00.123456: Critical error encountered!

Trying to use OldLogger directly (will fail or not what client expects):

Using OldLogger via Adapter:
Client: Writing some logs...
Adapter: Calling adaptee's log_message for level 1...
[OLD LOG][INFO] 2025-04-21T11:05:20.445267: Application started successfully.
Adapter: Calling adaptee's log_message for level 2...
[OLD LOG][WARNING] 2025-04-21T11:05:20.445295: Configuration file not found, using defaults.
Adapter: Calling adaptee's log_message for level 3...
[OLD LOG][ERROR] 2025-04-21T11:05:20.445313: Critical error encountered!


Сложный пример: Адаптация формата данных (XML к JSON/Dict)
Представим, что наше приложение работает с данными в формате словарей Python (или JSON), но получает их от внешнего сервиса в виде XML.

In [2]:
import xml.etree.ElementTree as ET
import json

# --- Adaptee ---
class XmlDataProvider:
    """Сервис, предоставляющий данные в формате XML."""
    def get_data_xml(self) -> str:
        # Имитация получения XML от внешнего источника
        return """
        <data>
            <item id="1">
                <name>Laptop</name>
                <price currency="USD">1200</price>
            </item>
            <item id="2">
                <name>Keyboard</name>
                <price currency="USD">75</price>
            </item>
        </data>
        """

# --- Target Interface (неявный) ---
# Клиент ожидает объект с методом get_data() -> list[dict]
# Пример ожидаемого формата: [{'id': '1', 'name': 'Laptop', 'price': '1200', 'currency': 'USD'}, ...]

# --- Adapter ---
class XmlToDictAdapter:
    """Адаптер, конвертирующий XML данные в список словарей."""
    def __init__(self, xml_provider: XmlDataProvider):
        self._adaptee = xml_provider

    def get_data(self) -> list[dict]:
        print("Adapter: Fetching XML data from adaptee...")
        xml_string = self._adaptee.get_data_xml()
        print("Adapter: Parsing XML and converting to list of dicts...")
        root = ET.fromstring(xml_string)

        data_list = []
        for item_element in root.findall('item'):
            item_data = {'id': item_element.get('id')} # Получаем атрибут
            name_element = item_element.find('name')
            if name_element is not None:
                item_data['name'] = name_element.text

            price_element = item_element.find('price')
            if price_element is not None:
                item_data['price'] = price_element.text
                item_data['currency'] = price_element.get('currency') # Получаем атрибут вложенного тега

            data_list.append(item_data)

        print("Adapter: Conversion complete.")
        return data_list

# --- Client ---
def process_inventory(data_source):
    """Клиентский код, который обрабатывает данные инвентаря (ожидает list[dict])."""
    print("\nClient: Processing inventory data...")
    inventory_data = data_source.get_data() # Использует Target интерфейс

    if not inventory_data:
        print("Client: No inventory data received.")
        return

    total_value = 0
    print("Client: Inventory items:")
    for item in inventory_data:
        print(f"  - ID: {item.get('id')}, Name: {item.get('name')}, Price: {item.get('price')} {item.get('currency')}")
        try:
            # Простая логика обработки - суммирование цен (если они есть и числовые)
            if item.get('price') and item.get('currency') == 'USD':
                total_value += float(item['price'])
        except (ValueError, TypeError):
            print(f"    Warning: Could not parse price for item ID {item.get('id')}")

    print(f"Client: Estimated total inventory value (USD): {total_value:.2f}")


# --- Использование ---
xml_service = XmlDataProvider()
# process_inventory(xml_service) # Ошибка, т.к. у xml_service нет метода get_data()

adapter = XmlToDictAdapter(xml_service)
process_inventory(adapter) # Теперь работает!

# Вывод:
# Adapter: Fetching XML data from adaptee...
# Adapter: Parsing XML and converting to list of dicts...
# Adapter: Conversion complete.
#
# Client: Processing inventory data...
# Client: Inventory items:
#   - ID: 1, Name: Laptop, Price: 1200 USD
#   - ID: 2, Name: Keyboard, Price: 75 USD
# Client: Estimated total inventory value (USD): 1275.00


Client: Processing inventory data...
Adapter: Fetching XML data from adaptee...
Adapter: Parsing XML and converting to list of dicts...
Adapter: Conversion complete.
Client: Inventory items:
  - ID: 1, Name: Laptop, Price: 1200 USD
  - ID: 2, Name: Keyboard, Price: 75 USD
Client: Estimated total inventory value (USD): 1275.00
